# Deploy trained XGBoost model to SageMaker Endpoint
---

This second notebook is to demonstrate on how to build **inference script** and host the **trained model** on SageMaker endpoint. I will utilize the *real-time* endpoint for this demonstration.

<div class="alert alert-block alert-info">
    This notebook has been tested and run on <b>Python 3 (Data Science 2.0)</b> kernel and on <b>ml.t3.medium</b> instance type.
</div>

## Setup

In [2]:
!pip install --upgrade pip sagemaker boto3 xgboost --quiet
!pip install pyxlsb --quiet

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.27.153 requires botocore==1.29.153, but you have botocore 1.31.18 which is incompatible.
awscli 1.27.153 requires PyYAML<5.5,>=3.10, but you have pyyaml 6.0.1 which is incompatible.
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussi

In [3]:
import xgboost as xgb
import logging
import sagemaker
import boto3 

# define the logger
logger = logging.getLogger()
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%I:%M:%S')
logger.setLevel(logging.INFO)

# version checking
logger.info(f'sagemaker => {sagemaker.__version__}')
logger.info(f'boto3 => {boto3.__version__}')
logger.info(f'xgboost => {xgb.__version__}')

02:43:10 INFO:sagemaker => 2.174.0
02:43:10 INFO:boto3 => 1.28.18
02:43:10 INFO:xgboost => 1.7.6


Define some parameters

In [4]:
REGION = boto3.Session().region_name
ROLE = sagemaker.get_execution_role()
sess = sagemaker.Session()
BUCKET_NM = sess.default_bucket()  # Set this to your own bucket name
PREFIX = 'byom/xgboost-model/'

## Deploy to SageMaker
---

Prior to deployment, I will specify the entry point for the data and the model.

Generally, we need to specify **model_fn**, **input_fn**, **predict_fn**, and **output_fn** within the `inference.py`. Alternatively, we can specify **transform_fn** to replace input_fn, predict_fn, and output_fn.


In [5]:
%%writefile code/inference.py
import json
from json import JSONEncoder
import os
import pickle as pkl
import numpy as np
import sagemaker_xgboost_container.encoder as xgb_encoders
import boto3
import pandas as pd
from io import StringIO

class NumpyArrayEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()

        return JSONEncoder.default(self, obj)


def model_fn(model_dir):
    """
    Deserialize and return fitted model.
    """
    model_file = "banking_loan_classifier.pkl"
    print('--- model_fn() ---')
    print(f'model dir => {model_dir}')
    print(f'mode file => {model_file}')
    booster = pkl.load(open(os.path.join(model_dir, model_file), "rb"))
    return booster


def transform_fn(model, request_body, content_type, accept_type):
    print('--- transform_fn() ---')
    print(f'content_type => {content_type}')
    print(f'accept_type => {accept_type}')
    print('--- Convert the input data type to dataframe ---\n')
    
    data = StringIO(request_body)
    cols = ['ID', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 
            'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'MARKER']
    _req_body = pd.read_csv(data, sep=",", header=None)
    _req_body.columns = cols
    print(f'The initial input shape => {_req_body.shape}')
    
    print('--- Input Transformation ---\n')
    _req_body.drop(['ID', 'MARKER'], axis=1, inplace=True)  # drop ID, and target
    _req_body.drop(['B', 'E', 'F'], axis=1, inplace=True)  # drop from correlation
    _req_body.rename(columns={'I': 'Sex'}, inplace=True)
    _req_body.rename(columns={'K': 'Region'}, inplace=True)
    _req_body.rename(columns={'M': 'Job_title'}, inplace=True)
    _req_body.rename(columns={'N': 'Education'}, inplace=True)
    _req_body.rename(columns={'O': 'Marriage'}, inplace=True)
    _req_body.rename(columns={'P': 'Children'}, inplace=True)
    _req_body.rename(columns={'Q': 'Property'}, inplace=True)
    _req_body.rename(columns={'S': 'Employment_status'}, inplace=True)
    _req_body = pd.get_dummies(_req_body, drop_first=True)
    print(f'The final input shape => {_req_body.shape}')

    print('--- Model Prediction ---\n')
    pred = model.predict(_req_body)
    print(f'Sample prediction => {pred[:5]}')
    print(f'Data type of prediction => {type(pred)}')
    
    _data = {'predictions': pred}  # we can enhance this by predict_proba and pass it in the JSON as well
    _out = json.dumps(_data, cls=NumpyArrayEncoder)
    return _out

Overwriting code/inference.py


In [6]:
!rm -f model.tar.gz
!tar -czvf model.tar.gz banking_loan_classifier.pkl code/inference.py

banking_loan_classifier.pkl
code/inference.py


Upload the model tarball with entry point `inference.py` to Amazon S3 bucket.

In [7]:
import boto3
import os 

BUCKET_NM = 'ml-ai-demo-th'
PREFIX = 'byom/xgboost-model'
obj = open("model.tar.gz", "rb")
key = os.path.join(PREFIX, "model.tar.gz")
logger.info(key)
boto3.Session().resource("s3").Bucket(BUCKET_NM).Object(key).upload_fileobj(obj)

02:43:11 INFO:byom/xgboost-model/model.tar.gz


In [8]:
from sagemaker.xgboost.model import XGBoostModel

model_data = f's3://{BUCKET_NM}/{key}'
logger.info(model_data)

# Create model object before deploying
trained_xgb = XGBoostModel(
    model_data=model_data,
    role=ROLE,
    entry_point="code/inference.py",
    framework_version="1.7-1",
)

02:43:11 INFO:s3://ml-ai-demo-th/byom/xgboost-model/model.tar.gz


In [9]:
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

predictor = trained_xgb.deploy(
    initial_instance_count=1,
    instance_type="ml.c5.xlarge",
    serializer=CSVSerializer(),
    deserializer=JSONDeserializer(),
)

02:43:11 INFO:Ignoring unnecessary instance type: ml.c5.xlarge.
02:43:12 INFO:Creating model with name: sagemaker-xgboost-2023-08-03-14-43-12-245
02:43:12 INFO:Creating endpoint-config with name sagemaker-xgboost-2023-08-03-14-43-12-889
02:43:13 INFO:Creating endpoint with name sagemaker-xgboost-2023-08-03-14-43-12-889


----!

## Validate the endpoint
---

To validate the endpoint, I will use the same `Test.xlsb` file and send it to the **real-time** endpoint from previous step.

In [10]:
import pandas as pd

test_file_nm = 'Test.xlsb'
test_df = pd.read_excel(io=f'data/{test_file_nm}')
logger.info(f'Testing data shape => {test_df.shape}')

02:46:00 INFO:Testing data shape => (38405, 26)


In [12]:
rt_pred = predictor.predict(
    test_df
)

Below cell is just to check the output.

In [13]:
from sklearn.metrics import classification_report

print(
    classification_report(rt_pred['predictions'], test_df['MARKER'])
)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     38314
           1       0.07      0.11      0.08        91

    accuracy                           0.99     38405
   macro avg       0.53      0.55      0.54     38405
weighted avg       1.00      0.99      1.00     38405



## Delete model and endpoint
---

If we are no longer used the endpoint, we should delete it.

In [14]:
predictor.delete_model()
predictor.delete_endpoint()

02:47:22 INFO:Deleting model with name: sagemaker-xgboost-2023-08-03-14-43-12-245
02:47:22 INFO:Deleting endpoint configuration with name: sagemaker-xgboost-2023-08-03-14-43-12-889
02:47:22 INFO:Deleting endpoint with name: sagemaker-xgboost-2023-08-03-14-43-12-889
